In [1]:
import numpy as np
import plotly.graph_objects as go

In [2]:
evals_history = np.load('evals_history.npy')[:, :]
evals_history.shape

(1080, 100)

In [3]:
risks_history = np.load('risks_history.npy')[:, :]
risks_history.shape

(1080, 100)

In [4]:
# get the last positive number from each row of evals_history numpy array
lasts = []
for eval in evals_history:
    # get last positive number from the numpy array
    only_nonzero = eval[eval != 0]
    if len(only_nonzero) == 0:
        last = 0
    else:
        last = only_nonzero[-1]
        
    lasts.append(last)

In [5]:
def rbf1(risks):
    prod_risk = 0
    for i in range(len(risks)):
        prod_risk = prod_risk + (np.power(0.99, i) * (risks[i]/10))

    if prod_risk < 0.01:
        prod_risk = 0.01

    slc = (1 - prod_risk) / ((prod_risk))

    # overriting rbf_delta
    rbf_delta = 1.0

    rbf = rbf_delta * slc

    return rbf

In [6]:
def rbf2(risks):
    prod_risk = 0
    for i in range(len(risks)):
        prod_risk = prod_risk + (np.power(0.99, i) * (risks[i]))

    slc = prod_risk

    # overriting rbf_delta
    rbf_delta = 1.0

    rbf = rbf_delta * slc

    return rbf

In [7]:
def rbf(evals):
    max_risk = 0
    for i in range(len(evals)):
        max_risk = max_risk + (np.power(0.99, i) * (evals[i]))

    slc = max_risk

    # overriting rbf_delta
    rbf_delta = 0.01

    rbf = 1 + rbf_delta * slc

    return rbf

In [8]:
def ser(risks):
    prod_safety = 1
    for i in range(len(risks)):
        prod_safety = prod_safety + (1 - (risks[i]))    

    ser = prod_safety

    return ser

In [9]:
# Teste para ver impactos de risco = 0
def ser(risks):

    risks = risks[(risks != 0) & (risks != 1)]

    prod_safety = 1
    for risk in risks[-5:]:
        if risk < 1:
            prod_safety = prod_safety * (1 - (risk/10))
        #print(prod_safety) 

    #         risco         /  segurança
    ser = (1 - prod_safety) / prod_safety

    return ser

In [10]:
sers = np.apply_along_axis(ser, 1, risks_history)
rbfs = np.apply_along_axis(rbf, 1, evals_history)

# plot sers and rbfs
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(sers)), y=sers, mode='lines', name="SER"))
fig.add_trace(go.Scatter(x=np.arange(len(rbfs)), y=rbfs, mode='lines', name="RBF"))
fig.add_trace(go.Scatter(x=np.arange(len(lasts)), y=lasts, mode='lines', name="E()"))
fig.update_layout(title='SER and RBF over time', xaxis_title='Time', yaxis_title='SER/RBF')
fig.show()

durante a simulacao ele encontra um estado de falha, precisa jogar o risk pra +infinito e remover o histórico automaticamente

In [11]:
risks_history[997]

array([0.93589741, 0.93630576, 0.93630576, 0.93630576, 0.93630576,
       0.93630576, 0.93630576, 0.93670887, 0.93670887, 0.93670887,
       0.93670887, 0.93670887, 0.93670887, 0.93670887, 0.93670887,
       0.93670887, 0.93670887, 0.93670887, 0.93710691, 0.93710691,
       0.93710691, 0.93710691, 0.93710691, 0.93710691, 0.93710691,
       0.93710691, 0.93710691, 0.93710691, 0.93710691, 0.93710691,
       0.93827158, 0.93827158, 0.93827158, 0.93827158, 0.93827158,
       0.93827158, 0.93827158, 0.93827158, 0.93827158, 0.93827158,
       0.93827158, 0.93827158, 0.93827158, 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [12]:
#plot all evals
fig = go.Figure()
for i in range(1):
    fig.add_trace(go.Scatter(x=np.arange(len(evals_history[i])), y=evals_history[i]/10, mode='lines'))
fig.update_layout(title='Eval over time', xaxis_title='Time', yaxis_title='Eval')
fig.show()


In [13]:
#plot all risks
fig = go.Figure()
for i in range(100):
    fig.add_trace(go.Scatter(x=np.arange(len(risks_history[i])), y=risks_history[i], mode='lines'))
fig.update_layout(title='Risk over time', xaxis_title='Time', yaxis_title='Risk')
fig.show()

In [14]:
#plot last positives
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(lasts)), y=lasts, mode='lines'))
fig.update_layout(title='Last eval over time', xaxis_title='Time', yaxis_title='E()')
fig.show()

In [15]:
def reward(evals):
    episode_reward = 0

    non_zero_evals = evals[evals != 0]

    for eval in non_zero_evals:
        episode_reward += eval

    episode_reward = episode_reward / len(non_zero_evals)

    return episode_reward

In [16]:
rewards = np.apply_along_axis(reward, 1, evals_history)

In [17]:
# plot rewards sum
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(rewards)), y=rewards, mode='lines'))
fig.update_layout(title='Rewards avg over time', xaxis_title='Time', yaxis_title='Rewards')
fig.show()

# MANIPULATING DATA

In [18]:
fake_evals_history = evals_history.copy()
fake_evals_history[200:600] = -evals_history[600:1000]

# get the last positive number from each row of evals_history numpy array
fake_lasts = []
for eval in fake_evals_history:
    # get last positive number from the numpy array
    only_nonzero = eval[eval != 0]
    if len(only_nonzero) == 0:
        last = 0
    else:
        last = only_nonzero[-1]
        
    fake_lasts.append(last)


# recalculate risks
fake_risks_history = []
for evals in fake_evals_history:
    risks = []
    for local_evaluation in evals:
        if local_evaluation >= 0:
            risk = 0.0
        else:
            risk = abs(local_evaluation)
        risks.append(risk)
    fake_risks_history.append(risks)



In [19]:
sers = np.apply_along_axis(ser, 1, fake_risks_history)
rbfs = np.apply_along_axis(rbf, 1, fake_evals_history)

# plot sers and rbfs
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(sers)), y=sers, mode='lines', name="SER"))
fig.add_trace(go.Scatter(x=np.arange(len(rbfs)), y=rbfs, mode='lines', name="RBF"))
fig.add_trace(go.Scatter(x=np.arange(len(fake_lasts)), y=fake_lasts, mode='lines', name="E()"))
fig.update_layout(title='SER and RBF over time', xaxis_title='Time', yaxis_title='SER/RBF')
fig.show()

- enquanto E for positivo, SER será sempre zerado, entao de nada adianta RBF crescer
- quando E é negativo, entao RBF começa a baixar (comportamento esperado)
- quando E é negativo, SER começa a crescer
- na intersecçao RBF limita SER